### 공통 설정: 모듈 import, model, API KEY 등 설정

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# ChatOpenAI 모델을 Groq API 엔드포인트로 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7 
)

# OpenAI API 키 (Groq API 키로 사용) 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### 문제 2-1 : 콤마 구분 리스트 파서 활용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser


print("\n--- 한국 유명 장소/활동 추천기를 시작합니다 ---")

# 1. CommaSeparatedListOutputParser 인스턴스 생성
# 이 파서는 LLM의 문자열 출력을 콤마로 구분된 리스트(Python list)로 변환해줍니다.
output_parser = CommaSeparatedListOutputParser()

# 2. ChatPromptTemplate 정의
# LLM에게 특정 분야에 대한 한국의 유명한 장소/활동 5가지를 콤마로 구분하여 출력하도록 지시합니다.
# 시스템 메시지에 출력 형식을 명확히 지정하여 파서가 올바르게 작동하도록 돕습니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 한국의 다양한 분야 전문가입니다. 사용자가 입력한 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마(,)로 구분된 리스트 형태로 추천해주세요. 예시: [장소1, 장소2, 장소3, 장소4, 장소5]\n\n{format_instructions}"),
    ("human", "관심 분야: {topic}")
])

# format_instructions를 프롬프트에 추가하여 파서의 지시사항을 LLM에 전달합니다.
# 이는 LLM이 파서가 기대하는 형식으로 응답하도록 유도합니다.
prompt_with_parser_instructions = prompt.partial(format_instructions=output_parser.get_format_instructions())

# 3. 체인 연결
# 프롬프트, LLM, 그리고 CommaSeparatedListOutputParser를 LCEL로 연결합니다.
recommendation_chain = prompt_with_parser_instructions | llm | output_parser

# 4. 사용자 입력 받기 및 실행
def recommend_korean_items(topic_input):
    print(f"\n입력 관심 분야: \"{topic_input}\"")
    print("\n--- 추천 결과 ---")
    recommended_list = recommendation_chain.invoke({"topic": topic_input})
    print(recommended_list)
    print(f"추천 개수: {len(recommended_list)}") # 리스트 형태로 잘 파싱되었는지 확인

# 예시 실행
user_topic = input("관심 있는 한국 분야를 입력하세요 (예: 음식, 스포츠, 영화): ")
recommend_korean_items(user_topic)

### 문제 2-2 : 영화 리뷰 감정 분석기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.enum import EnumOutputParser
from langchain.output_parsers import OutputFixingParser
from enum import Enum

print("\n--- 영화 리뷰 감정 분석기를 시작합니다 ---")

# 1. 감정 분류를 위한 Enum 클래스 정의
class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# 2. EnumOutputParser 인스턴스 생성
# target_enum에 위에서 정의한 Sentiment Enum 클래스를 전달합니다.
enum_parser = EnumOutputParser(enum=Sentiment)

# 3. OutputFixingParser와 EnumOutputParser 결합 (선택 사항이지만 추천)
# LLM이 예상치 못한 형식으로 응답했을 때, OutputFixingParser가 LLM에게 재시도하여 올바른 형식으로 수정하도록 지시합니다.
parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)


# 4. ChatPromptTemplate 정의
# LLM에게 영화 리뷰를 분석하여 정의된 Enum 값 중 하나로 응답하도록 지시합니다.
# {format_instructions}를 포함하여 파서의 기대 출력 형식을 LLM에 명확히 전달합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 리뷰의 감정을 정확하게 분석하는 전문가입니다. 주어진 리뷰의 감정을 '긍정', '부정', '보통' 중 하나로 분류하세요.\n\n{format_instructions}"),
    ("human", "영화 리뷰: {review}")
])

# format_instructions를 프롬프트에 추가하여 파서의 지시사항을 LLM에 전달합니다.
prompt_with_parser_instructions = prompt.partial(format_instructions=parser.get_format_instructions())


# 5. 체인 연결
# 프롬프트, LLM, 그리고 (OutputFixingParser를 포함한) EnumOutputParser를 LCEL로 연결합니다.
sentiment_analysis_chain = prompt_with_parser_instructions | llm | parser

# 6. 테스트 리뷰 예시
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "정말 기대했는데 실망이 컸어요. 다시 보고 싶지 않습니다.",
    "인생 영화 등극! 이 영화를 안 본 사람은 후회할 거예요.",
    "초반은 지루했지만, 후반부 반전이 인상 깊었어요." # 애매한 경우 테스트
]

# 7. 실행 및 결과 출력
print("--- 영화 리뷰 감정 분석 결과 ---")
for i, review in enumerate(test_reviews):
    print(f"\n[{i+1}] 리뷰: \"{review}\"")
    try:
        sentiment = sentiment_analysis_chain.invoke({"review": review})
        print(f"   감정: {sentiment.value}") # Enum 값의 실제 문자열 출력
        print(f"   타입: {type(sentiment)}") # 반환된 객체의 타입 확인 (Enum 멤버인지)
    except Exception as e:
        print(f"   감정 분석 실패: {e}")
        print("   LLM이 예상치 못한 형식으로 응답했거나, OutputFixingParser도 실패했습니다.")

### 문제 2-3: 학생 정보 구조화 시스템

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field # pydantic 관련 클래스 임포트
from typing import List

print("\n--- 학생 정보 구조화 시스템을 시작합니다 ---")

# 1. Pydantic BaseModel 정의
# 추출하고자 하는 정보의 구조와 타입을 정의합니다.
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 목록")
    goal: str = Field(description="학생의 미래 목표")

# 2. PydanticOutputParser 인스턴스 생성
# Pydantic 모델을 파서에 전달하여 LLM이 이 모델의 스키마에 맞춰 응답하도록 지시합니다.
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. ChatPromptTemplate 정의
# 시스템 메시지에서 LLM의 역할과 함께, Pydantic 파서가 생성한 형식 지시사항을 포함합니다.
# 이렇게 하면 LLM이 JSON 스키마에 맞춰 응답하도록 유도됩니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 학생의 자유 형식 자기소개에서 핵심 정보를 추출하여 구조화하는 전문가입니다. 다음 JSON 스키마에 따라 정보를 추출하세요:\n\n{format_instructions}"),
    ("human", "학생 자기소개: {self_introduction}")
])

# format_instructions를 프롬프트에 추가하여 파서의 지시사항(JSON 스키마)을 LLM에 전달합니다.
prompt_with_parser_instructions = prompt.partial(format_instructions=parser.get_format_instructions())

# 4. 체인 연결
# 프롬프트, LLM, 그리고 PydanticOutputParser를 LCEL로 연결합니다.
student_info_extraction_chain = prompt_with_parser_instructions | llm | parser

# 5. 테스트 자기소개 예시
test_introductions = [
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.",
    "제 이름은 이수진이고, 올해 25살입니다. 저는 디자인을 전공했으며, 그림 그리기와 사진 찍는 것을 즐겨 합니다. 언젠가는 저만의 스튜디오를 여는 것이 꿈입니다.",
    "저는 박준영이라고 합니다. 19세이고, 고등학교 졸업 후 바로 실용음악을 전공하려고 합니다. 기타 연주와 작곡이 취미이며, 유명한 밴드의 멤버가 되고 싶어요."
]

# 6. 실행 및 결과 출력
print("--- 학생 정보 추출 결과 ---")
for i, intro in enumerate(test_introductions):
    print(f"\n[{i+1}] 자기소개:\n\"{intro}\"")
    try:
        student_data = student_info_extraction_chain.invoke({"self_introduction": intro})
        print("   추출된 정보:")
        print(student_data.model_dump_json(indent=2)) # Pydantic 객체를 보기 좋게 JSON 문자열로 출력
        print(f"   타입: {type(student_data)}") # 반환된 객체의 타입 확인 (Pydantic 모델인지)
    except Exception as e:
        print(f"   정보 추출 실패: {e}")
        print("   LLM이 예상치 못한 형식으로 응답했거나, 파싱에 실패했습니다.")

### 문제 2-4 : 여행 계획 분석기

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import List

print("\n--- 여행 계획 분석기를 시작합니다 ---")

# 1. ResponseSchema 정의
# 추출하고자 하는 각 필드의 이름, 타입, 설명을 정의합니다.
response_schemas = [
    ResponseSchema(name="destination", description="여행지 이름", type="string"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일, 5일)", type="string"),
    ResponseSchema(name="budget", description="여행에 사용된 총 예산 (예: 30만원, 50달러)", type="string"),
    ResponseSchema(name="rating", description="여행에 대한 추천도 (1점에서 5점 사이의 숫자)", type="string"), # 숫자로 받아도 좋지만, 자유로운 텍스트 출력을 위해 string으로 유지
    ResponseSchema(name="activities", description="여행 중 주요 활동 목록", type="list")
]

# 2. StructuredOutputParser 인스턴스 생성
# 정의된 ResponseSchema 목록을 파서에 전달합니다.
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 3. ChatPromptTemplate 정의
# 시스템 메시지에서 LLM의 역할과 함께, StructuredOutputParser가 생성한 형식 지시사항을 포함합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 후기나 계획 텍스트에서 핵심 정보를 추출하여 구조화하는 전문가입니다. 다음 JSON 스키마에 따라 정보를 추출하고, 불필요한 서문이나 설명 없이 오직 JSON 객체만 반환하세요.\n\n{format_instructions}"),
    ("human", "여행 텍스트: {travel_text}")
])

# format_instructions를 프롬프트에 추가하여 파서의 지시사항(JSON 스키마)을 LLM에 전달합니다.
prompt_with_parser_instructions = prompt.partial(format_instructions=parser.get_format_instructions())

# 4. 체인 연결
# 프롬프트, LLM, 그리고 StructuredOutputParser를 LCEL로 연결합니다.
travel_analysis_chain = prompt_with_parser_instructions | llm | parser

# 5. 테스트 여행 텍스트 예시
test_travel_texts = [
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.",
    "이번 여름휴가에는 제주도로 4박 5일 갈 계획입니다. 예산은 100만원 정도 잡고 있어요. 한라산 등반, 서핑 배우기, 흑돼지 맛집 탐방이 주요 활동입니다. 벌써부터 기대되네요! 추천도는 당연히 5점 만점에 5점!",
    "친구들과 경주에 당일치기로 다녀왔어요. 비용은 거의 안 들었고, 불국사와 석굴암을 구경하고 황리단길에서 맛있는 거 먹었어요. 짧았지만 꽤 좋아서 3점 줍니다."
]

# 6. 실행 및 결과 출력
print("--- 여행 계획 분석 결과 ---")
for i, text in enumerate(test_travel_texts):
    print(f"\n[{i+1}] 여행 텍스트:\n\"{text}\"")
    try:
        travel_data = travel_analysis_chain.invoke({"travel_text": text})
        print("   추출된 정보:")
        # StructuredOutputParser는 기본적으로 딕셔너리를 반환합니다.
        # json.dumps를 사용하여 보기 좋게 출력합니다.
        import json
        print(json.dumps(travel_data, ensure_ascii=False, indent=2))
        print(f"   타입: {type(travel_data)}") # 반환된 객체의 타입 확인 (딕셔너리인지)
    except Exception as e:
        print(f"   정보 추출 실패: {e}")
        print("   LLM이 예상치 못한 형식으로 응답했거나, 파싱에 실패했습니다.")


--- 여행 계획 분석기를 시작합니다 ---
--- 여행 계획 분석 결과 ---

[1] 여행 텍스트:
"지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
   추출된 정보:
{
  "destination": "부산",
  "duration": "2박3일",
  "budget": "30만원",
  "rating": "4",
  "activities": [
    "해운대 바다구경",
    "자갈치시장에서 회 먹기",
    "감천문화마을 구경"
  ]
}
   타입: <class 'dict'>

[2] 여행 텍스트:
"이번 여름휴가에는 제주도로 4박 5일 갈 계획입니다. 예산은 100만원 정도 잡고 있어요. 한라산 등반, 서핑 배우기, 흑돼지 맛집 탐방이 주요 활동입니다. 벌써부터 기대되네요! 추천도는 당연히 5점 만점에 5점!"
   추출된 정보:
{
  "destination": "제주도",
  "duration": "4박5일",
  "budget": "100만원",
  "rating": "5",
  "activities": [
    "한라산 등반",
    "서핑 배우기",
    "흑돼지 맛집 탐방"
  ]
}
   타입: <class 'dict'>

[3] 여행 텍스트:
"친구들과 경주에 당일치기로 다녀왔어요. 비용은 거의 안 들었고, 불국사와 석굴암을 구경하고 황리단길에서 맛있는 거 먹었어요. 짧았지만 꽤 좋아서 3점 줍니다."
   추출된 정보:
{
  "destination": "경주",
  "duration": "당일치기",
  "budget": "거의 안 들었음",
  "rating": "3",
  "activities": [
    "불국사",
    "석굴암",
    "황리단길"
  ]
}
   타입: <class 'dict'>
